In [1]:
!git clone https://github.com/IrohXu/lanenet-lane-detection-pytorch.git

fatal: destination path 'lanenet-lane-detection-pytorch' already exists and is not an empty directory.


In [2]:
cd lanenet-lane-detection-pytorch

/home/jetson/lanenet_ws/src/lanenet-lane-detection-pytorch


In [17]:
import torch
import torch.nn as nn
import torch.nn.functional as F

from model.lanenet.loss import DiscriminativeLoss
from model.lanenet.backbone.UNet import UNet_Encoder, UNet_Decoder
from model.lanenet.backbone.ENet import ENet_Encoder, ENet_Decoder
#from model.lanenet.backbone.deeplabv3_plus.deeplabv3plus import Deeplabv3plus_Encoder, Deeplabv3plus_Decoder

DEVICE = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')


class LaneNet(nn.Module):
    def __init__(self, in_ch = 3, arch="ENet"):
        super(LaneNet, self).__init__()
        # no of instances for segmentation
        self.no_of_instances = 3  # if you want to output RGB instance map, it should be 3.
        print("Use {} as backbone".format(arch))
        self._arch = arch
        if self._arch == 'UNet':
            self._encoder = UNet_Encoder(in_ch)
            self._encoder.to(DEVICE)

            self._decoder_binary = UNet_Decoder(2)
            self._decoder_instance = UNet_Decoder(self.no_of_instances)
            self._decoder_binary.to(DEVICE)
            self._decoder_instance.to(DEVICE)
        elif self._arch == 'ENet':
            self._encoder = ENet_Encoder(in_ch)
            self._encoder.to(DEVICE)

            self._decoder_binary = ENet_Decoder(2)
            self._decoder_instance = ENet_Decoder(self.no_of_instances)
            self._decoder_binary.to(DEVICE)
            self._decoder_instance.to(DEVICE)
        elif self._arch == 'DeepLabv3+':
            self._encoder = Deeplabv3plus_Encoder()
            self._encoder.to(DEVICE)

            self._decoder_binary = Deeplabv3plus_Decoder(2)
            self._decoder_instance = Deeplabv3plus_Decoder(self.no_of_instances)
            self._decoder_binary.to(DEVICE)
            self._decoder_instance.to(DEVICE)
        else:
            raise("Please select right model.")

        self.relu = nn.ReLU().to(DEVICE)
        self.sigmoid = nn.Sigmoid().to(DEVICE)

    def forward(self, input_tensor):
        if self._arch == 'UNet':
            c1, c2, c3, c4, c5 = self._encoder(input_tensor)
            binary = self._decoder_binary(c1, c2, c3, c4, c5)
            instance = self._decoder_instance(c1, c2, c3, c4, c5)
        elif self._arch == 'ENet':
            c = self._encoder(input_tensor)
            binary = self._decoder_binary(c)
            instance = self._decoder_instance(c)
        elif self._arch == 'DeepLabv3+':
            c1, c2 = self._encoder(input_tensor)
            binary = self._decoder_binary(c1, c2)
            instance = self._decoder_instance(c1, c2)
        else:
            raise("Please select right model.")

        binary_seg_ret = torch.argmax(F.softmax(binary, dim=1), dim=1, keepdim=True)

        pix_embedding = self.sigmoid(instance)

        return {
            'instance_seg_logits': pix_embedding,
            'binary_seg_pred': binary_seg_ret,
            'binary_seg_logits': binary
        }

In [18]:
model = LaneNet(in_ch=3, arch="ENet")

Use ENet as backbone


In [25]:
print(model)

LaneNet(
  (_encoder): ENet_Encoder(
    (initial_block): InitialBlock(
      (conv): Sequential(
        (0): Conv2d(3, 13, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
        (1): BatchNorm2d(13, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): PReLU(num_parameters=1)
      )
      (maxpool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    )
    (bottleneck1_0): BottleneckModule(
      (activate): PReLU(num_parameters=1)
      (maxpool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (conv): Sequential(
        (0): Conv2d(16, 64, kernel_size=(2, 2), stride=(2, 2))
        (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): PReLU(num_parameters=1)
        (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (5): PReLU(num_p

In [26]:
from torchsummary import summary
summary(model, input_size = (3, 64, 64), batch_size = -1)

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 13, 32, 32]             364
       BatchNorm2d-2           [-1, 13, 32, 32]              26
             PReLU-3           [-1, 13, 32, 32]               1
         MaxPool2d-4            [-1, 3, 32, 32]               0
      InitialBlock-5           [-1, 16, 32, 32]               0
            Conv2d-6           [-1, 64, 16, 16]           4,160
       BatchNorm2d-7           [-1, 64, 16, 16]             128
             PReLU-8           [-1, 64, 16, 16]               1
            Conv2d-9           [-1, 64, 16, 16]          36,928
      BatchNorm2d-10           [-1, 64, 16, 16]             128
            PReLU-11           [-1, 64, 16, 16]               1
           Conv2d-12           [-1, 64, 16, 16]           4,160
      BatchNorm2d-13           [-1, 64, 16, 16]             128
            PReLU-14           [-1, 64,

In [21]:
cd src

/home/jetson/lanenet_ws/src


In [23]:
# Load the model and extract weights
import os 

checkpoint_path = os.path.join("model_weights_only.pth")
model.load_state_dict(torch.load(checkpoint_path), strict=False)

_IncompatibleKeys(missing_keys=['_encoder.initial_block.conv.0.bias', '_encoder.initial_block.conv.1.bias', '_encoder.initial_block.conv.1.running_mean', '_encoder.initial_block.conv.1.running_var', '_encoder.bottleneck1_0.conv.0.bias', '_encoder.bottleneck1_0.conv.1.bias', '_encoder.bottleneck1_0.conv.1.running_mean', '_encoder.bottleneck1_0.conv.1.running_var', '_encoder.bottleneck1_0.conv.3.bias', '_encoder.bottleneck1_0.conv.4.bias', '_encoder.bottleneck1_0.conv.4.running_mean', '_encoder.bottleneck1_0.conv.4.running_var', '_encoder.bottleneck1_0.conv.6.bias', '_encoder.bottleneck1_0.conv.7.bias', '_encoder.bottleneck1_0.conv.7.running_mean', '_encoder.bottleneck1_0.conv.7.running_var', '_encoder.bottleneck1_1.conv.0.bias', '_encoder.bottleneck1_1.conv.1.bias', '_encoder.bottleneck1_1.conv.1.running_mean', '_encoder.bottleneck1_1.conv.1.running_var', '_encoder.bottleneck1_1.conv.3.bias', '_encoder.bottleneck1_1.conv.4.bias', '_encoder.bottleneck1_1.conv.4.running_mean', '_encoder.b

In [52]:

import time
import os
import sys

import torch
from model.lanenet.LaneNet import LaneNet
from torch.autograd import Variable

from torchvision import transforms

from model.utils.cli_helper_eval import parse_args
from model.eval_function import Eval_Score

import numpy as np
from PIL import Image
import pandas as pd
import cv2

data_transform = transforms.Compose([
    transforms.Grayscale(3),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

def load_test_data(img_path, transform):
    img = Image.open(img_path)
    img = transform(img)
    return img

DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

img_path = os.path.join("lanenet-lane-detection-pytorch","data","source_image", "input.jpg")
resize_height = 0
resize_width = 0

model.to(DEVICE)

dummy_input = load_test_data(img_path, data_transform).to(DEVICE)
dummy_input = torch.unsqueeze(dummy_input, dim=0)
outputs = model(dummy_input)

instance_pred = torch.squeeze(outputs['instance_seg_logits'].detach().to('cpu')).numpy() * 255
binary_pred = torch.squeeze(outputs['binary_seg_pred']).to('cpu').numpy() * 255

color = "output.jpg"
val = instance_pred.transpose((1, 2, 0))

binary ="binary.jpg"
cv2.imwrite(path, val)
cv2.imwrite(binary, binary_pred)



True